---
# Ocular Torsion analysis based on eye tracker video
> Otero-Millan, J., Roberts, D. C., Lasker, A., Zee, D. S., & Kheradmand, A. (2015). Knowing what the brain is seeing in three dimensions: A novel, noninvasive, sensitive, accurate, and low-noise technique for measuring ocular torsion. Journal of Vision, 15(14), 1–15. https://doi.org/10.1167/15.14.11

## Set up Import & Function

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import random

def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output


def ConMask(canny, c, size):
    '''
    Find longest edge over a Region of Interest
    canny: image with Canny edges
    size: size of the ROI
    '''
    w,h=canny.shape
    ww,hh=size
    mask = np.zeros((w, h, 1), dtype='uint8')
    cv2.rectangle(mask,c,(c[0]+ww,c[1]+hh),255,-1)
    Ed = cv2.bitwise_and(canny,canny, mask=mask)
    cc = cv2.findContours(Ed,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnt = cc[0][np.argmax([len(i) for i in cc[0]])]
    
    return cnt

def ParaFit(x,y,width,buffer):
    '''
    Fit parabols in pixels to edges found with ConMask
    add buffer to make mask overlap image
    '''
    coef = np.polyfit(x,y,2)
    xTh = np.arange(0,width, dtype='int')
    pTh = np.poly1d(coef)
    yTh = pTh(xTh)
    
    i=int(buffer)
    yTh[yTh<50] = yTh[yTh<50]+i
    yTh[yTh>=50] = yTh[yTh>=50]-i
    
    return np.dstack((xTh,yTh.astype(int)))

def getGradientMagnitude(im, ratio=0.5):
    '''
    Get magnitude of gradient for given image
    '''
    ddepth = cv2.CV_32F
    ksize=3
    dx = cv2.Sobel(im, ddepth, 1, 0,ksize)
#     dx[dx>50]=0
    dy = cv2.Sobel(im, ddepth, 0, 1,ksize)
    dxabs = cv2.convertScaleAbs(dx)
    dyabs = cv2.convertScaleAbs(dy)
#     ratio=0.5
    mag = cv2.addWeighted(dxabs, ratio, dyabs, 1-ratio, gamma=0)
    
    return mag, dxabs, dyabs

# Various Constant
index = -1
thickness = 2
color1 = (0,0,255)
color2 = (255,0,0)
thresh=59
kernel = (7,7)
buffer=2
winSize=(360,60)
mir=25

In [3]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [4]:
readfile = '../video/GVS_2Hz.mov'

# __One Frame Analysis__

## Set up
gather information from Frame to analyze
constant for plotting
and basic image manipulation for analysis (gray image and blur)

In [5]:
cap = cv2.VideoCapture(readfile)
read_flag, frame = cap.read()
cap.release()

FR=frame.copy()

w, h = frame.shape[:2]

gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
blur = cv2.GaussianBlur(gray,kernel,0)
cl1 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
clahe = cl1.apply(gray)
# gNoise = cv2.GaussianBlur(sp_noise(gray,0.2),kernel,0)

if True:
    cv2.imshow('Gray',gray)
    cv2.imshow('CLAHE',clahe)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Eyelib detection and mask creation

 - find edges
 - define 4 windows for eyelid edge detection
 - find contours over the 4 windows
 - consolidate Top contours and Bottom contours
 - Fit parabols and create area of Interest

In [8]:
imOI = blur
edges = cv2.Canny(imOI,80,130,apertureSize = 3)

# region of interest (ROI)
ww=int(w/5)
hh=int(h/2-5)

if True:
    c1 = (int(w/4-ww),int(0))#Top-left
    c2 = (int(w/4-ww),int(h)-hh)#Bottom-left
    c3 = (int(3*w/4-ww),int(0))#Top-right
    c4 = (int(3*w/4-ww),int(h)-hh)#Bottom-right

## creating masks and contours over ROI
cnt1 = ConMask(edges, c1, (ww,hh))
cnt2 = ConMask(edges, c2, (ww,hh))
cnt3 = ConMask(edges, c3, (2*ww,hh))
cnt4 = ConMask(edges, c4, (2*ww,hh))

## X and Y of all contours
xTop = [k[0][0] for k in cnt1]
xTop.extend([k[0][0] for k in cnt3])
yTop = [k[0][1] for k in cnt1]
yTop.extend([k[0][1] for k in cnt3])

xBot = [k[0][0] for k in cnt2]
xBot.extend([k[0][0] for k in cnt4])
yBot = [k[0][1] for k in cnt2]
yBot.extend([k[0][1] for k in cnt4])

## polyfit
Top = ParaFit(xTop,yTop,w,buffer)
Bot = ParaFit(xBot,yBot,w,buffer)
EyeLid = np.vstack((Top.reshape(100,2),np.flip(Bot.reshape(100,2),0)))

## mask for EyeLid - eMask
eMask = np.zeros((w, h, 1), dtype='uint8')
cv2.fillPoly(eMask,[EyeLid],255)

## mask for Reflexions - rMask
negFR = cv2.bitwise_not(gray)
_, rMask = cv2.threshold(negFR,thresh,255,cv2.THRESH_BINARY)

# erosion for larger mask
kM = np.ones((buffer,buffer),np.uint8)
rMask = cv2.erode(rMask,kM,iterations = 1)

Mask = cv2.bitwise_and(eMask,rMask)

if True:
    ## draw the contours
    cv2.drawContours(FR,[EyeLid], index,color1,1)
    cv2.drawContours(FR, cnt1, index, color2, 2)
    cv2.drawContours(FR, cnt2, index, color2, 2)
    cv2.drawContours(FR, cnt3, index, color2, 2)
    cv2.drawContours(FR, cnt4, index, color2, 2)
    
    cv2.imshow('Eyelid Contours',FR)
    cv2.imshow('Reflexions',rMask)
    cv2.imshow('Mask',Mask)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Pupil detection

In [6]:
## Pupil detection
_, pupil = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY_INV)
contoursP, _ = cv2.findContours(pupil, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for c in contoursP:
    if len(c)<4:
        continue
    
    area = cv2.contourArea(c)
    
    if area<200:
        continue
    
    peri = cv2.arcLength(c, True)
    AA = area
    M = cv2.moments(c)# check moments of an image for centroids calculation
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.drawContours(FR,c,index,color1,1)

   
## Polar Transform
radius = np.array([cx, w-cx]).max()
Mag = w/np.log(radius)

# grayPL = cv2.logPolar(gray,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
grayPL = cv2.logPolar(clahe,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS) #  polar transform of high constrast image
rGray_PL = cv2.rotate(grayPL, cv2.ROTATE_90_CLOCKWISE) # rotated polar transformed image

maskPl = cv2.logPolar(Mask,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
rMask_PL = cv2.rotate(maskPl, cv2.ROTATE_90_CLOCKWISE)


## max Pupil size over planar representation - mPSz
ePL = cv2.Canny(rGray_PL,50,150,apertureSize = 3)
cc = cv2.findContours(ePL,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cntPl = cc[0][np.argmax([len(i) for i in cc[0]])]
mPSz = np.array([k[0][1] for k in cntPl]).max()

ResG = rGray_PL[mPSz+buffer:,:w]
ResM = rMask_PL[mPSz+buffer:,:w]

nResG = cv2.bitwise_and(ResG,ResG, mask=ResM)
# nResN = cv2.GaussianBlur(sp_noise(nResG,0.2),kernel,0)
# nResN = cv2.bitwise_and(nResN,nResN, mask=cv2.bitwise_not(ResM))


if True:
    cv2.drawMarker(FR,(cx,cy),color1)
    cv2.imshow('Pupil', FR)
    cv2.imshow('Polar transform Image', nResG)
    cv2.imshow('Polar transform Mask', ResM)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Sobel Filtering

In [11]:
Mg, dxabs, dyabs = getGradientMagnitude(nResG, ratio=0.5)
# Mg=cv2.Canny(nResG,85,95,apertureSize=5)
# Mg=nResG

## Add some noise to mask
ResN = cv2.GaussianBlur(sp_noise(Mg,0.5),(3,3),0)
ResN = cv2.bitwise_and(ResN,ResN, mask=cv2.bitwise_not(ResM))

nMg = cv2.bitwise_and(Mg,Mg,mask=ResM)
TPL = nMg+ResN
TPL = cv2.resize(TPL,winSize)

if True:
    plt.figure()
    plt.subplot(2,1,1)
    plt.imshow(nResG,cmap='gray')
    plt.subplot(2,1,2)
    plt.imshow(Mg,cmap='gray')
    
    plt.figure()
    plt.subplot(2,1,1)
    plt.imshow(ResN,cmap='gray')
    plt.subplot(2,1,2)
    plt.imshow(TPL,cmap='gray')


# Full Video Analysis

In [20]:
cap = cv2.VideoCapture(readfile)
Area_raw=[]

while True:
    read_flag, frame = cap.read()
    
    if not read_flag:
        break
    
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    FR = gray.copy()
    blur = cv2.GaussianBlur(gray,kernel,0)
    
    
    
    ## EyeLid detection
    edges = cv2.Canny(blur,80,130,apertureSize = 3)
    
    if True:
        cnt1 = ConMask(edges, c1, (ww,hh))
        cnt2 = ConMask(edges, c2, (ww,hh))
        cnt3 = ConMask(edges, c3, (2*ww,hh))
        cnt4 = ConMask(edges, c4, (2*ww,hh))

        xTop = [k[0][0] for k in cnt1]
        xTop.extend([k[0][0] for k in cnt3])
        yTop = [k[0][1] for k in cnt1]
        yTop.extend([k[0][1] for k in cnt3])

        xBot = [k[0][0] for k in cnt2]
        xBot.extend([k[0][0] for k in cnt4])
        yBot = [k[0][1] for k in cnt2]
        yBot.extend([k[0][1] for k in cnt4])
    
    Top = ParaFit(xTop,yTop,w,buffer)
    Bot = ParaFit(xBot,yBot,w,buffer)
    EyeLid = np.vstack((Top.reshape(100,2),np.flip(Bot.reshape(100,2),0)))
    
    
    
    ## mask for EyeLid - eMask
    eMask = np.zeros((w, h, 1), dtype='uint8')
    cv2.fillPoly(eMask,[EyeLid],255)
    ## mask for Reflexions - rMask
    negFR = cv2.bitwise_not(gray)
    _, rMask = cv2.threshold(negFR,thresh,255,cv2.THRESH_BINARY)
    # erosion for larger mask
    kM = np.ones((buffer,buffer),np.uint8)
    rMask = cv2.erode(rMask,kM,iterations = 1)
    Mask = cv2.bitwise_and(eMask,rMask)
    
    FR = cv2.bitwise_and(FR, FR, mask=Mask)

    
    
    ## Pupil detection
    _, pupil = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY_INV)
    contoursP, _ = cv2.findContours(pupil, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for c in contoursP:
        
        if len(c) < 4:
            continue
        
        area = cv2.contourArea(c)
        
        if area<200:
            continue
        
        peri = cv2.arcLength(c, True)
        M = cv2.moments(c)# check moments of an image for centroids calaculation
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.drawContours(FR,c,index,155)
        cv2.drawMarker(FR,(cx,cy),100)
        Area_raw.append(area)

        
    ## Log Planar transform
    radius = np.array([cx, w-cx]).max()
    Mag = w/np.log(radius)

    grayPL = cv2.logPolar(gray,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
    rGray_PL = cv2.rotate(grayPL, cv2.ROTATE_90_CLOCKWISE)

    maskPl = cv2.logPolar(Mask,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
    rMask_PL = cv2.rotate(maskPl, cv2.ROTATE_90_CLOCKWISE)

    ## max Pupil size over planar representation - mPSz
    ePL = cv2.Canny(rGray_PL,50,150,apertureSize = 3)
    cc = cv2.findContours(ePL,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cntPl = cc[0][np.argmax([len(i) for i in cc[0]])]
    mPSz = np.array([k[0][1] for k in cntPl]).max()

    ResG = rGray_PL[mPSz+buffer:,:w]
    ResM = rMask_PL[mPSz+buffer:,:w]

    nResG = cv2.bitwise_and(ResG,ResG, mask=ResM)
    
    ## Sobel Filtering
#     Mg,_,_ = getGradientMagnitude(nResG)
#     Mg=cv2.Canny(nResG,85,95,apertureSize=5)
    Mg = nResG
    
    ResN = cv2.GaussianBlur(sp_noise(Mg,0.1),(3,3),0)
    ResN = cv2.bitwise_and(ResN,ResN, mask=cv2.bitwise_not(ResM))
    nMg = cv2.bitwise_and(Mg,Mg,mask=ResM)
    IMG = nMg+ResN
    
    IMG = cv2.resize(IMG, winSize)

    if True:
        cv2.imshow("Edges", edges)
        cv2.imshow("Contours", FR)
        cv2.imshow('Planar transf', IMG)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()



AttributeError: 'NoneType' object has no attribute 'shape'

In [17]:
Fr = 120
T=np.linspace(1/Fr,len(Area_raw)/Fr,len(Area_raw))
T = T.tolist()
N=20
nData = np.convolve(Area_raw, np.ones((N,))/N, mode='same')
plt.plot(T,nData)
# plt.ylim(500,1500)

In [ ]:

toto = np.zeros((winSize[1], winSize[0]+2*mir), dtype='uint8')
toto[:,:mir]=IMG[:,-mir:]
toto[:,-mir:]=IMG[:,:mir]
toto[:,mir:-mir]=IMG

if True:
    res = cv2.matchTemplate(toto,TPL,cv2.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

cv2.imshow('TPL', TPL)
cv2.imshow('IMG', toto)
cv2.imshow('res', res)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
max_loc

In [18]:
res.shape

NameError: name 'res' is not defined

In [ ]:
plt.plot(res.ravel(),'.-')

In [ ]:
x=np.linspace(0,51,num=51)
y=res.ravel()
xth=np.linspace(0,51,101)
yth=np.interp(xth, x,y)

In [ ]:
plt.plot(x,y,'.-r')
plt.plot(xth,yth,'+k')